# LINE-1 IMPLEMENTATION

The original source code from the paper can be found here: [https://github.com/tangjianpku/LINE/tree/master](https://github.com/tangjianpku/LINE/tree/master)

This was also used slightly as help in this implementation.

In [1]:
import random
import time
from tqdm import tqdm
import networkx as nx
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import torch

from sklearn.model_selection import KFold, cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import MultiLabelBinarizer, normalize
from torch_geometric.utils import to_networkx, from_networkx
from torch_geometric.transforms import RandomLinkSplit

# Helper functions

These are the helper functions for LINE-1 algorithm. It uses an alias table that it samples from to make computations more efficient. Addtionally, it uses a negative sampling table for the same reasons.

Moreover, the code used for evaluating embeddings in node classification and link prediction can be found in this section.

In [2]:
def generate_alias(probs):
    """
    Generates an Alias table to sample from based on some probability distribution p
    :param probs: distribution to create the Alias table
    :return: A (Alias table)
    """
    l = len(probs)
    L, H = [], []
    
    for i, p in enumerate(probs):
        if p <= 1/l:
            L.append((i, p))
        else:
            H.append((i, p))
    
    A = []
    while len(L) > 0 and len(H) > 0:
        i, p_i = L.pop()
        h, p_h = H.pop()
        
        A.append((i, h, p_i))
        if p_h - p_i > 1/l:
            H.append((h, p_h - p_i))
        else:
            L.append((h, p_h - p_i))

    # Handling any remaining items
    while len(L) > 0:
        i, p = L.pop()
        A.append((i, i, 1/l))
    
    while len(H) > 0:
        h, p = H.pop()
        A.append((h, h, 1/l))
    return A


def sample_alias(A, l):
    """
    Samples an outcome from the alias table over a distribution p
    :param A: Alias table
    :param l: length of the probability distribution
    :return: an index based on the sampling outcome
    """
    draw = np.random.randint(l)
    i, h, p = A[draw]
    if np.random.rand() < l * p:
        return h
    else:
        return i


def init_neg_table(G, node2int, size):
    """
    Creates a table for the negative sampling of vertices according to vertex degrees 
    :param G: Graph from which to sample
    :param size: size of the table
    :return: Negative sampling table
    """
    degrees_prob = np.zeros(G.number_of_nodes())
    for node, d in G.degree:
        # P_n(v) ∝ d_v ** 0.75 from the paper
        degrees_prob[node2int[node]] = d ** 0.75
    
    Z = np.sum(degrees_prob)
    neg_table = np.zeros(int(size), dtype=np.uint32)
    
    neg_table_id = 0
    for i, p in enumerate(degrees_prob):
        portion_to_fill = np.round((p/Z) * size).astype(int)
        neg_table[neg_table_id:neg_table_id+portion_to_fill] = i
        neg_table_id += portion_to_fill
    return neg_table


# ---- PREDICTION TASKS ----
def node_classification(X, y, n_folds=5):
    """
    5-fold multi-label classification using one-vs-rest logistic regression
    :param X: source embeddings of a graph
    :param y: the labels for each node
    :param n_folds: number of folds for cross-validation
    :return: accuracy, f1 macro score, f1 micro score
    """
    model = LogisticRegression()
    ovr_model = OneVsRestClassifier(model)
    kf = KFold(n_splits=n_folds, shuffle=True)
    eval_scores = {'acc': 'accuracy', 'f1_macro': 'f1_macro', 'f1_micro': 'f1_micro'}
    results = cross_validate(ovr_model, X, y, cv=kf, scoring=eval_scores)
    acc, f1_macro, f1_micro = results['test_acc'].mean(), results['test_f1_macro'].mean(), results['test_f1_micro'].mean()
    return acc, f1_macro, f1_micro


def predict_link(u, v, embeddings):
    """
    Computes the normalized probability for an existing link between two nodes u and v based on the input
    embeddings.
    :param u: a node in the graph
    :param v: a node in the graph
    :param embeddings: trained embeddings
    :return: sigmoid normalized probability for the existence of a link
    """
    embedding1 = embeddings[u]
    embedding2 = embeddings[v]
    
    # Compute inner product (dot product)
    dot_product = np.dot(embedding1, embedding2)

    # Normalize by sigmoid function
    link_probability = 1/(1 + np.exp(-dot_product))
    return link_probability


def link_predictions(embeddings, edges, y_true):
    """
    Computes the ROC-AUC score for a given set of test edges based on the trained embeddings.
    :param embeddings: a models trained embeddings
    :param edges: test edges
    :param y_true: the labels for edges (1=true, 0=false)
    :return: the ROC-AUC score from predictions
    """
    predictions = []
    for edge in edges:
        predictions.append(predict_link(edge[0], edge[1], embeddings))
    return roc_auc_score(y_true, predictions) 


# reference: https://zqfang.github.io/2021-08-12-graph-linkpredict/
def train_test_split_graph(G):
    """
    Splits a Graph into a test and train set randomly to 80-20. The test split is balanced with negative edges sampled from random vertex pairs that have no edges between them. 
    While removing edges randomly, it makes sure that no vertex is isolated.
    :param G: a networkx graph to be split
    :return: the train-test split as torch geometrics graphs
    """
    data = from_networkx(G)
    try:
        data.y = data.group_belonging
    except:
        try:
            data.y = data.club  # this only happens with karate club
        except:
            print('No labels')  # only for youtube
    data.x = torch.arange(G.number_of_nodes()).unsqueeze(1)
    
    transform = RandomLinkSplit(num_val=0, num_test=0.5, is_undirected=True, add_negative_train_samples=False)
    train_data, _, test_data = transform(data)
    return train_data, test_data


# SILVIA'S METHOD
def multi_label_classification(X, y, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True)
    micro_f1_scores = []
    macro_f1_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Predictions
        model = OneVsRestClassifier(LogisticRegression())
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        micro_f1 = f1_score(y_test, y_pred, average='micro')
        macro_f1 = f1_score(y_test, y_pred, average='macro')

        micro_f1_scores.append(micro_f1)
        macro_f1_scores.append(macro_f1)
    
    # Calculate mean scores over all folds
    mean_micro_f1 = np.mean(micro_f1_scores)
    mean_macro_f1 = np.mean(macro_f1_scores)

    return mean_micro_f1, mean_macro_f1

# LINE-1 class

In [3]:
class Line:
    def __init__(self, G, dim, alpha, neg_samples, num_edge_samples):
        """
        Initializes a LINE-1 embedding algorithm
        :param G: Graph as a networkx object
        :param dim: dimension of embeddings
        :param alpha: learning rate
        :param neg_samples: number of negative samples in training
        :param num_edge_samples: number of edge samples used in training
        """
        self.neg_table_size = 1e8  # default, same as in the original source code
        self.num_nodes = G.number_of_nodes()
        self.num_edges = G.number_of_edges()
        self.node2int = {node: i for i, node in enumerate(G.nodes())}  # dictionary for easy access in arrays
        
        # Compute the probability weights for each edge
        self.edges = [[self.node2int[u], self.node2int[v]] for u, v in G.edges()]
        edges_prob = np.array([G[u][v].get("weight", 1) for u, v in G.edges()])
        self.edges_prob = edges_prob/np.sum(edges_prob)
        
        # Create sampling tables for the algorithm
        self.neg_table = init_neg_table(G, self.node2int, self.neg_table_size)
        self.alias_table = generate_alias(self.edges_prob)
        
        # hyperparameters
        self.dim = dim
        self.alpha_0 = alpha
        self.neg_samples = neg_samples
        self.T = num_edge_samples
        
        self.embeddings = (np.random.random((self.num_nodes, self.dim)) - 0.5)/self.dim
        
    def update(self, u_i, u_j, alpha, acc_gradient, local_label):
        """
        The update equation for the SGD in the LINE-1 algorithm
        :param u_i: source representation for vertex i
        :param u_j: source representation for vertex j
        :param acc_gradient: accumulated gradient
        :param local_label: label in the local graph (1=connected, 0=not connected)
        :return: 
        """
        f_line = 1/(1 + np.exp(-(u_i @ u_j)))  # sigmoid
        g = (local_label - f_line) * alpha
        
        acc_gradient += g * u_j
        u_j += g * u_i
        return 

    def train_line(self, file_name):
        """
        Trains a LINE-1 algorithm embeddings via Edge Sampling with a minibatch size of 1
        :return: trained embeddings
        """
        
        start = time.time()
        alpha = self.alpha_0
        for i in tqdm(range(int(self.T))):
            if i > 0:
                alpha = self.alpha_0*(1-i/self.T)
            
            edge = sample_alias(self.alias_table, self.num_edges)
            u, v = self.edges[edge]
            
            acc_gradient = np.zeros(self.dim)
            self.update(self.embeddings[u], self.embeddings[v], alpha, acc_gradient, local_label=1)
            
            for k in range(self.neg_samples):
                v = random.choice(self.neg_table)
                self.update(self.embeddings[u], self.embeddings[v], alpha, acc_gradient, local_label=0)
            
            self.embeddings[u] += acc_gradient

        norm_trained_embeddings = normalize(self.embeddings)
        np.save(f'./embeddings/{file_name}.npy', norm_trained_embeddings)
        self.embeddings = norm_trained_embeddings
        print(f"Training time: {time.time() - start}s")
        return norm_trained_embeddings
    

# Flickr Experiments

In [4]:
# Load the data
edges_path = './Flickr-dataset/data/edges.csv'
nodes_path = './Flickr-dataset/data/nodes.csv'
groups_path = './Flickr-dataset/data/groups.csv'
group_edges_path = './Flickr-dataset/data/group-edges.csv'

nodes_id = pd.read_csv(nodes_path, header=None, names=['id'])
groups_id = pd.read_csv(groups_path, header=None, names=['group'])
edges = pd.read_csv(edges_path, header=None, names=['id_1', 'id_2'])
user_group_membership = pd.read_csv(group_edges_path, header=None, names=['id', 'group'])

In [5]:
# Create a graph
G_flickr = nx.Graph()

# Add nodes to the graph
G_flickr.add_nodes_from(nodes_id['id'])

# Add edges to the graph
G_flickr.add_edges_from(edges[['id_1', 'id_2']].values)

In [6]:
# Create a dictionary to store groups for each ID
group_dict = {}

# Populate the group_dict
for _, row in user_group_membership.iterrows():
    user_id = row['id']
    group_id = row['group']

    # Check if the user_id is already in the dictionary
    if user_id in group_dict:
        group_dict[user_id].append(group_id)
    else:
        group_dict[user_id] = [group_id]

# Add group labels to the nodes
for user_id, groups in group_dict.items():
    nx.set_node_attributes(G_flickr, {user_id: groups}, 'group_belonging')

# Print basic graph information
print("Number of nodes:", G_flickr.number_of_nodes(), ' | ', "Number of edges:", G_flickr.number_of_edges())

Number of nodes: 80513  |  Number of edges: 5899882


In [7]:
# Find and preprocess labels for the graph
labels = []
for n in G_flickr.nodes:
    l = G_flickr.nodes[n].get('group_belonging')
    labels.append(l)

mlb = MultiLabelBinarizer()
preprocessed_labels = mlb.fit_transform(labels)

In [8]:
# hyperparameters from the paper
dim = 128
alpha = 0.025
neg_samples = 5
T = 1e9

model_flickr = Line(G_flickr, dim, alpha, neg_samples, T)

In [9]:
embeddings_model_flickr = model_flickr.train_line('flickr')

100%|██████████| 1000000000/1000000000 [7:26:15<00:00, 37347.26it/s]  


Training time: 26775.90695786476s


In [ ]:
# Load embeddings
embeddings_model_flickr = np.load('./embeddings/flickr.npy')

**Node Classification:**

In [10]:
N = 5
f1_macros, f1_micros = np.zeros(N), np.zeros(N)
for i in range(N):
    _, f1_macro, f1_micro = node_classification(embeddings_model_flickr, preprocessed_labels)
    f1_macros[i] = f1_macro
    f1_micros[i] = f1_micro
    print(f'Shuffle {i+1} --', 'F1 macro:', f1_macro, 'F1 micro:', f1_micro)

Shuffle 1 -- F1 macro: 0.1093505917150007 F1 micro: 0.24001922287709862
Shuffle 2 -- F1 macro: 0.11024349410182673 F1 micro: 0.24049589540655142
Shuffle 3 -- F1 macro: 0.11046383391444581 F1 micro: 0.24123100654959523
Shuffle 4 -- F1 macro: 0.11025984517029346 F1 micro: 0.24116316021219864
Shuffle 5 -- F1 macro: 0.1110781932341988 F1 micro: 0.24104611684485713


In [11]:
print('-----  Final results: -----')
print('F1 macro:', np.mean(f1_macros), 'F1 micro:', np.mean(f1_micros))

-----  Final results: -----
F1 macro: 0.11027919162715309 F1 micro: 0.24079108037806024


**Link Prediction:**

In [38]:
N = 5
roc_auc_scores = np.zeros(N)
for i in range(N):
    train_data, test_data = train_test_split_graph(G_flickr)

    # Prepare edges
    test_edges = test_data.edge_label_index.numpy().T
    y_true = test_data.edge_label.numpy()

    score = link_predictions(embeddings_model_flickr, test_edges, y_true)
    roc_auc_scores[i] = score
    print(f'Shuffle {i+1} --', 'ROC-AUC:', score)

Shuffle 1 -- ROC-AUC: 0.7757859136307986
Shuffle 2 -- ROC-AUC: 0.7760684936323055
Shuffle 3 -- ROC-AUC: 0.776027779504015
Shuffle 4 -- ROC-AUC: 0.7760370577166799
Shuffle 5 -- ROC-AUC: 0.775577489183476


In [39]:
print('-----  Final results: -----')
print('ROC-AUC:', np.mean(roc_auc_scores))

-----  Final results: -----
ROC-AUC: 0.7758993467334551


# YouTube Experiments

In [21]:
# Load the data
edges_path = './YouTube-dataset/data/edges.csv'
nodes_path = './YouTube-dataset/data/nodes.csv'
groups_path = './YouTube-dataset/data/groups.csv'
group_edges_path = './YouTube-dataset/data/group-edges.csv'

nodes_id = pd.read_csv(nodes_path, header=None, names=['id'])
groups_id = pd.read_csv(groups_path, header=None, names=['group'])
edges = pd.read_csv(edges_path, header=None, names=['id_1', 'id_2'])
user_group_membership = pd.read_csv(group_edges_path, header=None, names=['id', 'group'])

In [22]:
# Create a graph
G_YT = nx.Graph()

# Add nodes to the graph
G_YT.add_nodes_from(nodes_id['id'])

# Add edges to the graph
G_YT.add_edges_from(edges[['id_1', 'id_2']].values)

# Create a copy of the graph for link prediction before adding labels, since some of the nodes don't have labels the method crashes otherwise
G_YT2 = G_YT.copy()

In [23]:
# Create a dictionary to store groups for each ID
group_dict = {}

# Populate the group_dict
for _, row in user_group_membership.iterrows():
    user_id = row['id']
    group_id = row['group']

    # Check if the user_id is already in the dictionary
    if user_id in group_dict:
        group_dict[user_id].append(group_id)
    else:
        group_dict[user_id] = [group_id]

# Add group labels to the nodes
for user_id, groups in group_dict.items():
    nx.set_node_attributes(G_YT, {user_id: groups}, 'group_belonging')

# Print basic graph information
print("Number of nodes:", G_YT.number_of_nodes(), ' | ', "Number of edges:", G_YT.number_of_edges())

Number of nodes: 1138499  |  Number of edges: 2990443


In [35]:
# Find and preprocess labels for the graph, most nodes do not have labels so leave them out
labels = []
indices = []
for i, n in enumerate(G_YT.nodes):
    l = G_YT.nodes[n].get('group_belonging')
    if l != None:
        labels.append(l)
        indices.append(i)

mlb = MultiLabelBinarizer()
preprocessed_labels = mlb.fit_transform(labels)

In [25]:
# hyperparameters from the paper
dim = 128
alpha = 0.025
neg_samples = 5
T = 1e9

model_yt = Line(G_YT, dim, alpha, neg_samples, T)

In [26]:
embeddings_model_yt = model_yt.train_line('youtube')

100%|██████████| 1000000000/1000000000 [7:40:37<00:00, 36182.69it/s]  


Training time: 27638.943197250366s


In [ ]:
# Load embeddings
embeddings_model_yt = np.load('./embeddings/youtube.npy')

**Node Classification:**

In [36]:
N = 5
f1_macros, f1_micros = np.zeros(N), np.zeros(N)
for i in range(N):
    _, f1_macro, f1_micro = node_classification(embeddings_model_yt[indices], preprocessed_labels)
    f1_macros[i] = f1_macro
    f1_micros[i] = f1_micro
    print(f'Shuffle {i+1} --', 'F1 macro:', f1_macro, 'F1 micro:', f1_micro)

Shuffle 1 -- F1 macro: 0.21879560136393889 F1 micro: 0.2929703145996874
Shuffle 2 -- F1 macro: 0.21841685041915698 F1 micro: 0.29284233497228035
Shuffle 3 -- F1 macro: 0.2194178898023121 F1 micro: 0.2933688411087377
Shuffle 4 -- F1 macro: 0.2184629801304176 F1 micro: 0.29334369387037584
Shuffle 5 -- F1 macro: 0.21867739781644344 F1 micro: 0.2930418131645823


In [37]:
print('-----  Final results: -----')
print('F1 macro:', np.mean(f1_macros), 'F1 micro:', np.mean(f1_micros))

-----  Final results: -----
F1 macro: 0.2187541439064538 F1 micro: 0.2931133995431327


**Link Prediction:**

In [45]:
N = 5
roc_auc_scores = np.zeros(N)
for i in range(N):
    train_data, test_data = train_test_split_graph(G_YT2)

    # Prepare edges
    test_edges = test_data.edge_label_index.numpy().T
    y_true = test_data.edge_label.numpy()

    score = link_predictions(embeddings_model_yt, test_edges, y_true)
    roc_auc_scores[i] = score
    print(f'Shuffle {i+1} --', 'ROC-AUC:', score)

No labels
Shuffle 1 -- ROC-AUC: 0.593335804968768
No labels
Shuffle 2 -- ROC-AUC: 0.593483231118844
No labels
Shuffle 3 -- ROC-AUC: 0.5939454527615484
No labels
Shuffle 4 -- ROC-AUC: 0.5932828620257373
No labels
Shuffle 5 -- ROC-AUC: 0.5936624478545048


In [46]:
print('-----  Final results: -----')
print('ROC-AUC:', np.mean(roc_auc_scores))

-----  Final results: -----
ROC-AUC: 0.5935419597458805


# Reddit Experiments

In [4]:
from torch_geometric.datasets import Reddit
dataset = Reddit("./")
data = dataset[0]

In [5]:
# convert to networkx
G_reddit = to_networkx(data)
G_reddit = G_reddit.to_undirected()

# Create a dictionary to store groups for each ID
group_dict = {}

# Populate the group_dict
for i in range(len(data.y)):
    user_id = i
    group_id = int(data.y[i].numpy())

    # Check if the user_id is already in the dictionary
    if user_id in group_dict:
        if type(group_id) == int:
            group_dict[user_id].append(group_id)
        else:
            group_dict[user_id].extend(group_id)
    else:
        if type(group_id) == int:
            group_dict[user_id] = [group_id]
        else:
             group_dict[user_id] = list(group_id)

# Add group labels to the nodes
for user_id, groups in group_dict.items():
    nx.set_node_attributes(G_reddit, {user_id: groups}, 'group_belonging')

# Print basic graph information
print("Number of nodes:", G_reddit.number_of_nodes(), ' | ', "Number of edges:", G_reddit.number_of_edges())

Number of nodes: 232965  |  Number of edges: 57307946


In [14]:
# Find and preprocess labels for the graph
labels = []
for n in G_reddit.nodes:
    l = G_reddit.nodes[n].get('group_belonging')
    labels.append(l)

mlb = MultiLabelBinarizer()
preprocessed_labels = mlb.fit_transform(labels)

In [15]:
# hyperparameters from the paper
dim = 128
alpha = 0.025
neg_samples = 5
T = 1e9

model_reddit = Line(G_reddit, dim, alpha, neg_samples, T)

In [16]:
embeddings_model_reddit = model_reddit.train_line('reddit')

 50%|█████     | 502606923/1000000000 [10:37:36<10:30:59, 13137.70it/s]


KeyboardInterrupt: 

In [17]:
np.save(f'./embeddings/reddit.npy', normalize(model_reddit.embeddings))

In [6]:
# Load embeddings
embeddings_model_reddit = np.load('./embeddings/reddit.npy')

**Node Classification:**

In [19]:
N = 5
f1_macros, f1_micros = np.zeros(N), np.zeros(N)
for i in range(N):
    _, f1_macro, f1_micro = node_classification(embeddings_model_reddit, preprocessed_labels)
    f1_macros[i] = f1_macro
    f1_micros[i] = f1_micro
    print(f'Shuffle {i+1} --', 'F1 macro:', f1_macro, 'F1 micro:', f1_micro)

Shuffle 1 -- F1 macro: 0.8872435748014531 F1 micro: 0.9232672579502028
Shuffle 2 -- F1 macro: 0.88720976200274 F1 micro: 0.923153071842032
Shuffle 3 -- F1 macro: 0.8872037583157116 F1 micro: 0.9232989911232746
Shuffle 4 -- F1 macro: 0.887149947471468 F1 micro: 0.9232640658000217
Shuffle 5 -- F1 macro: 0.8873171961993054 F1 micro: 0.9232863279466331


In [20]:
print('-----  Final results: -----')
print('F1 macro:', np.mean(f1_macros), 'F1 micro:', np.mean(f1_micros))

-----  Final results: -----
F1 macro: 0.8872248477581357 F1 micro: 0.923253942932433


**Link Prediction:**

In [ ]:
N = 5
roc_auc_scores = np.zeros(N)
for i in range(N):
    train_data, test_data = train_test_split_graph(G_reddit)

    # Prepare edges
    test_edges = test_data.edge_label_index.numpy().T
    y_true = test_data.edge_label.numpy()

    score = link_predictions(embeddings_model_reddit, test_edges, y_true)
    roc_auc_scores[i] = score
    print(f'Shuffle {i+1} --', 'ROC-AUC:', score)

Shuffle 1 -- ROC-AUC: 0.9143625631022497
Shuffle 2 -- ROC-AUC: 0.9142920870122443


In [ ]:
print('-----  Final results: -----')
print('ROC-AUC:', np.mean(roc_auc_scores))